**<font color = black size=6>实验十:EM算法</font>**

In [45]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

**<font color = blue size=4>第一部分:实验任务</font>**

<span style="color:purple">=======  
现在给定了一个班级中所有同学的身高数据文件(height.csv)，但不知道各个学生的性别。假设男生身高服从一高斯分布$N_1(\mu_1,\sigma_1^2)$，女生身高服从另一高斯分布$N_2(\mu_2,\sigma_2^2)$，我们可以写出一个混合高斯模型:$x\sim\alpha_1 N_1(\mu_1,\sigma_1^2)+\alpha_2 N_2(\mu_2,\sigma_2^2)$。请使用EM算法完成对该混合高斯分布的估计(即求出对参数$\alpha_1,\alpha_2,\mu_1,\sigma_1,\mu_2,\sigma_2$的估计值)。我们简化地记$\theta_1=(\alpha_1,\mu_1,\sigma_1)$, $\theta_2=(\alpha_2,\mu_2,\sigma_2)$</span>

<span style="color:purple">该数据集(height.csv)特征信息只包括了1个特征，即学生的身高。我们沿用理论课PPT的设置，将隐变量$z_i$按照男生$z_i=1$、女生$z_i=2$的形式进行描述</span>

**<font color = black size=4>E步（Expectation Step）</font>**

<span style="color:purple">1) 将数据集'height.csv'载入并转换为你需要的格式</span>

In [46]:
# 导入数据集，并将二维数组展平
D = pd.read_csv('data.csv').values.flatten()
print(D)


[170.26930751 179.8860453  173.68132906 ... 163.56147404 163.49419524
 171.81596527]


<span style="color:purple">2)初始化  
初始化$t=0$时的参数($\alpha_1(0)$, $\alpha_2(0)$, $\mu_1(0)$, $\mu_2(0)$, $\sigma_1(0)$, $\sigma_2(0)$). </span>

In [47]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]

# param = [0, 0, 0, 0, 0, 0]


<span style="color:purple">3)编写函数P(x,parameter,z)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D,计算每个样本$x_i$的$P(x_i,z_i|\theta)$.</span>

<span style="color:purple">.  
当$z_i=1$时,$$P(x_i,z_i|\theta)=\alpha_1(t)f_1(x_i|\theta_1(t))$$
    当$z_i=2$时,$$P(x_i,z_i|\theta)=\alpha_2(t)f_2(x_i|\theta_2(t))$$</span>

<span style="color:purple"> .   
其中$f_1(x_i|\theta_1(t))$为样本$x_i$在模型$N_1$中的概率密度,公式如下:
    $$f(x_i|\theta_1(t))=\frac{1}{{\sqrt{2\pi}\sigma_1}} e^{-\frac{(x_i-\mu_1)^2}{2\sigma_1^2}}$$</span>

In [48]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def P(x, para, z):
    alpha1, alpha2, mu1, mu2, sigma1, sigma2 = para

    if z == 1:
        f1_x = (1 / (math.sqrt(2 * math.pi) * sigma1)) * math.exp(-(x - mu1) ** 2 / (2 * sigma1 ** 2))
        prob = alpha1 * f1_x
    elif z == 2:
        f2_x = (1 / (math.sqrt(2 * math.pi) * sigma2)) * math.exp(-(x - mu2) ** 2 / (2 * sigma2 ** 2))
        prob = alpha2 * f2_x
    else:
        print("error in func P")
    
    return prob
        

<span style="color:purple">4)编写函数Y(x,parameter,z)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D,计算每个样本$x_i$的$y_{1,i}=P((z_i=1)|x_i,\theta)$和$y_{2,i}=P((z_i=2)|x_i,\theta)$.  
公式如下:  
</span>

$$P((z_i=1)|x_i,\theta) = \frac{\alpha_1(t)f_1(x_i|\theta_1(t))}{\alpha_1(t)f_1(x_i|\theta_1(t))+\alpha_2(t)f_2(x_i|\theta_2(t))}$$  
$$P((z_i=2)|x_i,\theta) = \frac{\alpha_2(t)f_2(x_i|\theta_2(t))}{\alpha_1(t)f_1(x_i|\theta_1(t))+\alpha_2(t)f_2(x_i|\theta_2(t))}$$

In [49]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def Y(x, para, z):

    if z == 1:
        deno =  P(x, para, z) + P(x, para, z+1)
    elif z == 2:
        deno =  P(x, para, z) + P(x, para, z-1)
    else:
        print("error in func Y")
    
    prob = P(x, para, z) / deno

    return prob



<span style="color:purple">5)编写函数Q(x,parameter)  
 计算对数似然函数在该分布和基于$\theta(t)$下的期望值$Q(\theta)$.单个样本的期望值公式如下:$$E_{z_i}logP(x_i,z_i|\theta)=y_{1,i}log(\alpha_1(t)f_1(x_i|\theta_1(t)))+y_{2,i}log(\alpha_2(t)f_2(x_i|\theta_2(t)))$$</span>

In [50]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def Q(D, para):
    
    # 初始化期望值
    exp = 0

    # 累加期望值
    for d in D:
        log_likelihood_1 = math.log(P(d, para, 1))
        log_likelihood_2 = math.log(P(d, para, 2))
        exp += Y(d, para, 1) * log_likelihood_1 + Y(d, para, 2) * log_likelihood_2

    # 返回期望值
    return exp

**<font color = black size=4>M步 (Maximization Step)</font>**

<span style="color:purple">6)编写函数alpha_expection(D,parameter)  
 给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D，计算第$(t+1)$轮的$(\alpha_1(t+1),\alpha_2(t+1))$的更新值.
</span>

$$\alpha_1(t+1)=\frac{\sum_{i=1}^m{y_{1,i}}}{m}$$  
$$\alpha_2(t+1)=\frac{\sum_{i=1}^m{y_{2,i}}}{m}$$

In [51]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def alpha_expection(D, para):
    # 初始化
    sum_1 = 0
    sum_2 = 0

    # 遍历累加
    for d in D:
        sum_1 += Y(d, para, 1)
        sum_2 += Y(d, para, 2)
    
    # 计算alpha
    alpha_1 = sum_1 / len(D)
    alpha_2 = sum_2 / len(D)

    # 返回参数
    return alpha_1, alpha_2 


<span style="color:purple">7)编写函数mu_expection(D,parameter)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D，计算第$(t+1)$轮的$(\mu_1(t+1),\mu_2(t+1))$的更新值.
</span>

$$\mu_1(t+1)=\frac{\sum_{i=1}^m{y_{1,i}x_i}}{\sum_{i=1}^m{y_{1,i}}}$$
$$\mu_2(t+1)=\frac{\sum_{i=1}^m{y_{2,i}x_i}}{\sum_{i=1}^m{y_{2,i}}}$$

In [52]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def mu_expection(D, para):

    # 初始化
    sum_1 = sum_2 = sum_3 = sum_4 = 0

    # 遍历累加
    for d in D:
        sum_1 += Y(d, para, 1) * d
        sum_2 += Y(d, para, 1)
        sum_3 += Y(d, para, 2) * d
        sum_4 += Y(d, para, 2)

    # 计算mu
    mu1 = sum_1 / sum_2
    mu2 = sum_3 / sum_4
    
    # 返回参数
    return mu1, mu2  
    

<span style="color:purple">8)编写函数
sigma_expection(D,parameter,mu_next_1,mu_next_2)
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D，计算第$(t+1)$轮的$(\sigma_1(t+1),\sigma_2(t+1))$的更新值.
</span>

$$\sigma_1(t+1)=\sqrt{\frac{\sum_{i=1}^m{y_{1,i}(x_i-\mu_1(t+1))^2}}{\sum_{i=1}^m{y_{1,i}}}}$$
$$\sigma_2(t+1)=\sqrt{\frac{\sum_{i=1}^m{y_{2,i}(x_i-\mu_2(t+1))^2}}{\sum_{i=1}^m{y_{2,i}}}}$$

In [53]:
# parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def sigma_expection(D, para, mu_next_1, mu_next_2):
    # 初始化
    sum_1 = sum_2 = sum_3 = sum_4 = 0

    # 遍历累加
    for d in D:
        sum_1 += Y(d, para, 1) * (d - mu_next_1)**2
        sum_2 += Y(d, para, 1)
        sum_3 += Y(d, para, 2) * (d - mu_next_2)**2
        sum_4 += Y(d, para, 2)
    
    # 更新sigma
    sigma1 = math.sqrt(sum_1 / sum_2)
    sigma2 = math.sqrt(sum_3 / sum_4)

    return sigma1, sigma2

**<font color = black size=4>E步和M步的迭代过程</font>**

<span style="color:purple">9) 利用前面编写的函数完成EM算法的迭代过程，直至达到收敛要求。请至少完成【3次】不同的初始值下的迭代过程，并比较选出最好的。  
    收敛要求给出如下几种参考:  
    1.迭代轮数达到指定轮数;  
    2.每轮参数更新的差值小于阈值.</span>

<img src='./EM Algorithm Pseudocode.png'>

.  我们给出这个数据集的正确相关信息作为参考:$\theta_1:(\alpha_1=0.625,\mu_1=175,\sigma_1=4)$,$\theta_2:(\alpha_2=0.375,\mu_2=165,\sigma_2=6)$

In [54]:
# 设置参数
conv_threshold = 1e-5
max_iterations = 500

def Jud(x1, x1_old, x2, x2_old):
    if abs(x1 - x1_old) < conv_threshold and abs(x2 - x2_old) < conv_threshold:
        return True
    else:
        return False

# 构造EM模型
def EM_model(D, para):
    # 进行初始化
    iteration = 0

    while iteration < max_iterations:
        
        # 获取原参数
        a1_old, a2_old = para[0], para[1]
        # m1_old, m2_old = para[2], para[3]
        # s1_old, s2_old = para[4], para[5]

        # 计算新参数
        a1, a2 = alpha_expection(D, para)
        m1, m2 = mu_expection(D, para)
        s1, s2 = sigma_expection(D, para, m1, m2)

        # 判断每轮参数更新的差值是否小于阈值
        if Jud(a1, a1_old, a2, a2_old):
            break

        # 更新参数与迭代轮次
        para = [a1, a2, m1, m2, s1, s2]

        iteration += 1
    
    return para, iteration


In [55]:

# 初始化
init_param = [
    [0.5, 0.3, 165, 155, 4, 4],
    [0.55, 0.35, 170, 157, 4.5, 4.5],
    [0.6, 0.4, 175, 160, 5, 5],
    [0.625, 0.375, 175, 165, 4, 6],
    [0.6, 0.4, 180, 160, 5, 5],
    [0.65, 0.45, 180, 170, 5, 5],
    [0.7, 0.5, 185, 175, 6, 6]
]

# 训练模型
for i, param in enumerate(init_param):
    parameters, iteration = EM_model(D, param)
    print(f"Initialization {i+1}:")
    print("Train epoches:", iteration)
    print("Parameters:", parameters)


Initialization 1:
Train epoches: 383
Parameters: [0.6305261097891913, 0.36947389021080823, 175.37840914131485, 164.99846191545026, 4.024156606039094, 5.627846335915675]
Initialization 2:
Train epoches: 373
Parameters: [0.630524491970305, 0.36947550802969464, 175.3784180047124, 164.9984922402896, 4.024152607222392, 5.627859210332814]
Initialization 3:
Train epoches: 344
Parameters: [0.6305300815336361, 0.3694699184663641, 175.3783873813691, 164.99838746771857, 4.024166423282355, 5.627814729103191]
Initialization 4:
Train epoches: 12
Parameters: [0.6285483935201539, 0.3714516064798451, 175.38665026251877, 165.03978263324893, 4.021589700651456, 5.648674414418014]
Initialization 5:
Train epoches: 266
Parameters: [0.6305237275785418, 0.36947627242145836, 175.37842219249083, 164.99850656823583, 4.024150717862136, 5.6278652932643265]
Initialization 6:
Train epoches: 441
Parameters: [0.6290784054402864, 0.370921594559714, 175.3863114191577, 165.02557261092605, 4.020592299568402, 5.639354950209

**<font color = blue size=4>第二部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、实验报告截止日期： 【11月24日 14:20】
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验十(EM算法)的实验报告上交地址:https://send2me.cn/9UjusmMn/S_Cz3o_FpKQEsA  

三、课堂课件获取地址:https://www.jianguoyun.com/p/DZKTh-IQp5WhChiIn6gFIAA  
实验内容获取地址:https://www.jianguoyun.com/p/DbCHB9wQp5WhChiKn6gFIAA